In [269]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter,MarkdownHeaderTextSplitter
from langchain.chains import VectorDBQA
from langchain.document_loaders import TextLoader,DirectoryLoader
from sentence_transformers import SentenceTransformer
import requests
import json
import os 
import re
import numpy as np
import faiss
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.vectorstores.faiss import FAISS
import pandas as pd

In [270]:
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings,HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import HuggingFaceHub
from tqdm.auto import tqdm

In [271]:
def get_pdf_text(pdf_docs):
    texts = []
    attribute = []
    for pdf in tqdm(pdf_docs):
        text = ''
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text += page.extract_text()
        texts.append(text)
    return texts
    
def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=512,
        chunk_overlap=128,
        length_function=len
            )
    chunks = text_splitter.split_documents(text)
    return chunks
    


In [272]:

def load_faiss_index():
    with open('chunks_gb.pickle', 'rb') as handle:
        chunks = pickle.load(handle)

    model = SentenceTransformer('BAAI/bge-m3',device="cpu")
    embeddings = np.load('embs-gb.npy')

    index = faiss.IndexFlatL2(embeddings.shape[1])   # build the index
    index.add(embeddings)     

    return chunks, model, index, embeddings

def get_relevant_documents(question, chunks, model, index, embeddings):
    k = 3 #NUM of retrieval candidates
    e = model.encode(question)
    dist, Idx = index.search(e.reshape(1,-1), k)
    retrievals = [chunks[i] for i in Idx.flatten()]
    return retrievals  

In [251]:
def generate_mixtral_response(question, chunks, model, index, embeddings):
    rets = get_relevant_documents(question, chunks, model, index, embeddings)
    metadatas = [list(item.metadata.keys())[0] for item in rets]
    
    # Improved string formatting
    sources_text = ' \n\n '.join([f'ИСТОЧНИК {i+1}: {ret.page_content}' for i, ret in enumerate(rets)])
    
    promptstring = (
        f"Вы программист, который точно отвечает на вопросы пользователей на темы, связанные с программированием. "
        f"Используя информацию, содержащуюся в пронумерованных ИСТОЧНИКАХ после слова ТЕКСТ, "
        f"ответьте на вопрос, заданный после слова ВОПРОС. "
        f"Если в тексте нет информации, необходимой для ответа, ответьте «Недостаточно информации для ответа». "
        f"Структурируйте свой ответ и отвечайте шаг за шагом. При ответе с использованием информации из текста, "
        f"используйте ссылки в скобках, содержащие номер ИСТОЧНИКА с релевантной информацией.\n"
        f"ТЕКСТ:\n{sources_text}\nВОПРОС:\n{question}"
    )

    #print(promptstring)
    endpoint = 'https://api.together.xyz/v1/chat/completions'
    res = requests.post(endpoint, json={
    "model": "mistralai/Mixtral-8x7B-Instruct-v0.1",
    "max_tokens": 2048,
    "prompt": f"[INST] {promptstring}  [/INST]",
    "temperature": 0.4,
    "top_p": 0.7,
    "top_k": 5,
    "repetition_penalty": 1,
    "stop": [
        "[/INST]",
        "</s>"
    ],
    "repetitive_penalty": 1,
    "update_at": "2024-02-25T16:35:32.555Z"
}, headers={
    "Authorization": "Bearer ",
})
    print(dict(json.loads(res.content))['usage']['total_tokens'])
    return dict(json.loads(res.content))['choices'][0]['message']['content'],metadatas,rets


In [316]:
def generate_mixtral_question(sources_text, n):
        
    # Improved string formatting
    sources_text = sources_text
    
    promptstring = (
        f"Вы программист, который точно отвечает на вопросы пользователей на темы, связанные с программированием. "
        f"Сформулируй, пожалуйста {n} экземпляра вопросов к тексту после слова ТЕКСТ:  "
        f"ТЕКСТ:\n{sources_text}"
    )

    #print(promptstring)
    endpoint = 'https://api.together.xyz/v1/chat/completions'
    res = requests.post(endpoint, json={
    "model": "mistralai/Mixtral-8x7B-Instruct-v0.1",
    "max_tokens": 2048,
    "prompt": f"[INST] {promptstring}  [/INST]",
    "temperature": 0.4,
    "top_p": 0.7,
    "top_k": 5,
    "repetition_penalty": 1,
    "stop": [
        "[/INST]",
        "</s>"
    ],
    "repetitive_penalty": 1,
    "update_at": "2024-02-25T16:35:32.555Z"
}, headers={
    "Authorization": "Bearer ",
})
    #print(dict(json.loads(res.content))['usage']['total_tokens'])
    return dict(json.loads(res.content))['choices'][0]['message']['content']

In [317]:
def generate_mixtral_comment(question, answer, chunks, model, index, embeddings):
    rets = get_relevant_documents(question, chunks, model, index, embeddings)
    metadatas = [list(item.metadata.keys())[0] for item in rets]
    
    # Improved string formatting
    sources_text = ' \n\n '.join([f'ИСТОЧНИК {i+1}: {ret.page_content}' for i, ret in enumerate(rets)])
    
    promptstring = (
        f"Вы программист, который точно отвечает на вопросы пользователей на темы, связанные с программированием. "
        f"Используя информацию, содержащуюся в пронумерованных ИСТОЧНИКАХ после слова ТЕКСТ, "
        f"оцените правильность высказывания, приведенного после слова ОТВЕТ, как ответ на вопрос, заданный после слова ВОПРОС. "
        f"Если в тексте нет информации, необходимой для ответа, ответьте «0*». " #Недостаточно информации для ответа
        f"Если высказывание после слова ОТВЕТ правильное, ответьте «1*». Если высказывание после слова ОТВЕТ неправильное, ответьте «0*».\n "
        f"дополнительно используйте ссылки в скобках, содержащие номер ИСТОЧНИКА с релевантной информацией.\n"
        f"ТЕКСТ:\n{sources_text}\nВОПРОС:\n{question}\nОТВЕТ:{answer}"
    )

    #print(promptstring)
    endpoint = 'https://api.together.xyz/v1/chat/completions'
    res = requests.post(endpoint, json={
    "model": "mistralai/Mixtral-8x7B-Instruct-v0.1",
    "max_tokens": 2048,
    "prompt": f"[INST] {promptstring}  [/INST]",
    "temperature": 0.4,
    "top_p": 0.7,
    "top_k": 5,
    "repetition_penalty": 1,
    "stop": [
        "[/INST]",
        "</s>"
    ],
    "repetitive_penalty": 1,
    "update_at": "2024-02-25T16:35:32.555Z"
}, headers={
    "Authorization": "Bearer ",
})
    #print(dict(json.loads(res.content))['usage']['total_tokens'])
    return dict(json.loads(res.content))['choices'][0]['message']['content'],metadatas,rets


In [329]:
#create a full file with materials for question preparation from genereated xls files
path = os.getcwd()
data_path = path+'\\data\\'
fnames = []
for file in os.listdir(data_path):
    if '.xls' in file:
        fnames.append(data_path + file)
        
print(fnames)
df = [pd.read_excel(fname, sheet_name=None) for fname in fnames[:1]]
df = df[0]['Sheet1']
df['Курс'] = df.loc[0,'Курс']
df['Урок №'] = df.loc[0,'Урок №']
df['Название урока'] = df.loc[0,'Название урока']
for fname in fnames[1:]:
    df_tmp = pd.read_excel(fname, sheet_name=None)['Sheet1']
    df_tmp['Курс'] = df_tmp.loc[0,'Курс']
    df_tmp['Урок №'] = df_tmp.loc[0,'Урок №']
    df_tmp['Название урока'] = df_tmp.loc[0,'Название урока']
 
    df = pd.concat([df,df_tmp], axis = 0, ignore_index = True)
materials = df.copy()

['C:\\Users\\andre\\OneDrive\\Desktop\\0-Rojects\\2024 GB CFO\\data\\introduction_less_1.xlsx', 'C:\\Users\\andre\\OneDrive\\Desktop\\0-Rojects\\2024 GB CFO\\data\\introduction_less_2.xlsx', 'C:\\Users\\andre\\OneDrive\\Desktop\\0-Rojects\\2024 GB CFO\\data\\introduction_less_3.xlsx', 'C:\\Users\\andre\\OneDrive\\Desktop\\0-Rojects\\2024 GB CFO\\data\\introduction_less_4.xlsx', 'C:\\Users\\andre\\OneDrive\\Desktop\\0-Rojects\\2024 GB CFO\\data\\introduction_less_5.xlsx', 'C:\\Users\\andre\\OneDrive\\Desktop\\0-Rojects\\2024 GB CFO\\data\\introduction_less_6.xlsx', 'C:\\Users\\andre\\OneDrive\\Desktop\\0-Rojects\\2024 GB CFO\\data\\process_lection_1.xlsx', 'C:\\Users\\andre\\OneDrive\\Desktop\\0-Rojects\\2024 GB CFO\\data\\process_lection_2.xlsx', 'C:\\Users\\andre\\OneDrive\\Desktop\\0-Rojects\\2024 GB CFO\\data\\process_lection_3.xlsx', 'C:\\Users\\andre\\OneDrive\\Desktop\\0-Rojects\\2024 GB CFO\\data\\process_lection_4.xlsx', 'C:\\Users\\andre\\OneDrive\\Desktop\\0-Rojects\\2024 GB 

In [331]:
materials = materials[materials['Урок №']=='Урок 5']

In [332]:
#select No of questions to each paragraph
n=3
questions_df = pd.DataFrame(columns = ['Курс','Урок №','Название урока','Название абзаца','Questions'])	
for i in tqdm(range(len(materials))):
    text = materials.iloc[i,4]
    tmp = pd.DataFrame(columns = ['Курс','Урок №','Название урока','Название абзаца','Questions'])
    #print(tmp)
    tmp.loc[0,'Курс'] = materials.iloc[i,0]
    tmp.loc[0,'Урок №'] = materials.iloc[i,1]
    tmp.loc[0,'Название урока'] = materials.iloc[i,2]
    tmp.loc[0,'Название абзаца'] = materials.iloc[i,3]
    #print(tmp)
    questions = generate_mixtral_question(materials.iloc[3,4], n)
    start = 0
    for k in range(n):
        try:
            start = questions.find('.',start+1)
            end = questions.find('?',start+1)
            question = questions[start+2:end+1]
            tmp['Questions'] = question
            questions_df = pd.concat([questions_df, tmp], axis = 0, ignore_index = True)
        except:
            continue
questions_df    

    

  0%|          | 0/41 [00:00<?, ?it/s]

,Курс,Урок №,Название урока,Название абзаца,Questions
0,Введение в тестирование,Урок 5,Виды тестирования,Функциональное тестирование,Какие виды знаний и умений необходимы тестиров...
1,Введение в тестирование,Урок 5,Виды тестирования,Функциональное тестирование,На каких этапах разработки приложения может бы...
2,Введение в тестирование,Урок 5,Виды тестирования,Функциональное тестирование,Какие преимущества и недостатки имеет тестиров...
3,Введение в тестирование,Урок 5,Виды тестирования,Три ящика тестирования,Какие виды знаний и умений необходимы тестиров...
4,Введение в тестирование,Урок 5,Виды тестирования,Три ящика тестирования,На каких этапах разработки приложения может пр...
...,...,...,...,...,...
118,Тестирование мобильных приложений,Урок 5,Нефункциональное тестирование,Используемая литература,На каких этапах разработки приложения может бы...
119,Тестирование мобильных приложений,Урок 5,Нефункциональное тестирование,Используемая литература,Как выбираются входные значения для тестирован...
120,Тестирование мобильных приложений,Урок 5,Нефункциональное тестирование,Дополнительные материалы,Какие виды знаний и понимания требуются тестир...
121,Тестирование мобильных приложений,Урок 5,Нефункциональное тестирование,Дополнительные материалы,На каких уровнях тестирования наиболее часто п...


In [333]:
questions_df.to_csv('questions_db.csv', index = False)